In [ ]:
#예제 10-1
open('cafe.txt', 'w', encoding='utf_8').write('café')

4

In [ ]:
#예제 10-2
open('cafe.txt').read() #파일을 읽을 때 인코딩 유형을 지정하지 않으면 파이썬은 기본 인코딩(windows 1252)를 이용한다. 이에 따른 문자 변형이 존재할 수 있다.

'café'

In [ ]:
#예제 11
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """
my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [ ]:
#예제 12-1
from unicodedata import normalize
s1 = 'café' #é은 e와 '가 결합된 결합문자이다.
s2 = 'cafe\u0301' #출력 상 café라고 나옴
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [ ]:
#예제 12-2
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [ ]:
#예제 12-3
len(normalize('NFC', s1)), len(normalize('NFC', s2)) #정규화 방식 C(NFC) 코드 포인트를 조합해서 가장 짧은 동일 문자열을 생성 => s2가 café가 된다

(4, 4)

In [ ]:
#예제 12-4
len(normalize('NFD', s1)), len(normalize('NFD', s2)) #정규화 방식 N(NFD) 코드 포인트를 분해해서 조합된 문자를 기본 문자와 별도의 결합문자로 분리

(5, 5)

In [ ]:
#예제 12-5
normalize('NFC', s1) == normalize('NFC', s2)


True

In [ ]:
#예제 12-6
normalize('NFD', s1) == normalize('NFD', s2)

True

In [ ]:
#예제 13-1
from unicodedata import normalize, name
ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [ ]:
#예제 13-2
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [ ]:
#예제 13-3
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [ ]:
#예제 13-4
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [ ]:
#예제 14-1
eszett = 'ß'
#기본문자는 샤프에스이다.
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [ ]:
#예제 14-2
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
#casefold를 통해 문자가 바뀐 경우다. 여기선 ss로 변환된다.
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

In [ ]:
#예제 15-1
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [ ]:
#예제 15-2
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold())
nfc_equal(s1, s2)

True

In [ ]:
#예제 15-3
nfc_equal('A', 'a')

False

In [ ]:
#예제 16-1
import unicodedata
import string

def shave_marks(txt):
    norm_txt = unicodedata.normalize('NFD', txt) #NFD는 조합 문자를 기본 문자와 결합 문자로 분할하는 기능이 있다.
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c)) #결합 문자를 제외한 기본 문자들로 재구성된 문자열을 만든다.
    return unicodedata.normalize('NFC', shaved)

In [ ]:
#예제 16-2
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [ ]:
#예제 16-3
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

'Ζεφυρος, Zefiro'

In [ ]:
#예제 16-4
shave_marks('Straße')

'Straße'

In [ ]:
#예제 16-5
shave_marks('cafè')

'cafe'

In [ ]:
#예제 17
def shave_marks_latin(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue
        keepers.append(c)
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)

In [ ]:
#예제 18-1
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")
multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)

def dewinize(txt):
    return txt.translate(multi_map)

def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))
    no_marks = no_marks.replace('ß', 'ss')
    return unicodedata.normalize('NFKC', no_marks)


In [ ]:
#예제 18-2
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [ ]:
#예제 18-3
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

In [ ]:
#예제 19
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [ ]:
#예제 20-1
!pip install pyuca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.1 MB/s eta 0:00:00


In [ ]:
#예제 20-2
import pyuca
coll = pyuca.Collator()
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [ ]:
#예제 21
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),
          char.center(6),
          're_dig' if re_digit.match(char) else '-',
          'isdig' if char.isdigit() else '-',
          'isnum' if char.isnumeric() else '-',
          'isdecimal' if char.isdecimal() else '-',
          sep='\t')


U+0031	  1   	re_dig	isdig	isnum	isdecimal
U+00bc	  ¼   	-	-	isnum	-
U+00b2	  ²   	-	isdig	isnum	-
U+0969	  ३   	re_dig	isdig	isnum	isdecimal
U+136b	  ፫   	-	isdig	isnum	-
U+216b	  Ⅻ   	-	-	isnum	-
U+2466	  ⑦   	-	isdig	isnum	-
U+2480	  ⒀   	-	-	isnum	-
U+3285	  ㊅   	-	-	isnum	-


In [ ]:
#예제 22
import re
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
              "as 1729 = 1³ + 12³ = 9³ + 10³.")
text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))
print('  bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print('  str  :', re_words_str.findall(text_str))
print('  bytes:', re_words_bytes.findall(text_bytes))

Text
  'Ramanujan saw ௧௭௨௯as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


In [74]:
#예제 23-1
import os

os.listdir(".")

['.config',
 '.ipynb_checkpoints',
 'dummy',
 'fluent_python',
 'cafe.txt',
 'sample_data']

In [75]:
#예제 23-2
os.listdir(b'.')

[b'.config',
 b'.ipynb_checkpoints',
 b'dummy',
 b'fluent_python',
 b'cafe.txt',
 b'sample_data']

In [76]:
#예제 24-1
os.listdir('./fluent_python')

['.ipynb_checkpoints', 'digits-of-(pi).txt']

In [77]:
#예제 24-2
os.listdir(b'./fluent_python')

[b'.ipynb_checkpoints', b'digits-of-(pi).txt']

In [78]:
#예제 24-3
pi_name_bytes = os.listdir(b'.')[1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'.ipynb_checkpoints'